# 1) Installing dependencies:-

In [1]:
%pip install -q google-genai chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2) Import and config:-

In [13]:
from google import genai
import chromadb
from chromadb.utils.embedding_functions import EmbeddingFunction
from dotenv import load_dotenv

GOOGLE_API_KEY = "API_KEY"  # https://aistudio.google.com/api-keys

genai_client = genai.Client(api_key=GOOGLE_API_KEY)
chroma_client = chromadb.Client()


# 3) Adding Your documents:- 

In [3]:
documents = [
    "Kopala guruviah, would snatch your phone and your soul for his daughter. His cabin is AA 105",
    "Enigma, (best club out there) has a cute penguin mascot named pebbles ",
    "Best drink you can get in college's nescafe's hot milk for 59rs"
]

# 4) Embedding:-
         > converts text to something mathematical that we can do math on and, vectors

In [4]:
class GoogleEmbedding(EmbeddingFunction):
    def __init__(self, client: genai.Client, model_name: str = "gemini-embedding-001"):
        self.client = client
        self.model_name = model_name

    def __call__(self, input):
        texts = input if isinstance(input, list) else [input]
        response = self.client.models.embed_content(
            model=self.model_name,
            contents=texts,
        )
        return [item.values for item in response.embeddings]

embedding_fn = GoogleEmbedding(genai_client)
collection = chroma_client.get_or_create_collection(
    name="documents",
    embedding_function=embedding_fn,
)

for i, doc in enumerate(documents):
    collection.add(
        documents=[doc],
        ids=[f"doc_{i}"],
    )

# 5) Querying:-
            > via cosine similarity

In [5]:
def search_documents(query, n_results=2):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    return results['documents'][0]

# 6) LLM call:-

In [6]:
def generate_response(query, context):
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {query}

Answer:"""
    response = genai_client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=prompt,
    )
    return response.text

In [7]:
def rag_query(query):
    relevant_docs = search_documents(query)
    context = "\n".join(relevant_docs)
    answer = generate_response(query, context)
    
    print(f"Question: {query}\n")
    print(f"Context:\n{context}\n")
    print(f"Answer:\n{answer}")
    
    return answer

# examples:-

In [8]:
rag_query("best club in college?")

Question: best club in college?

Context:
Best drink you can get in college's nescafe's hot milk for 59rs
Enigma, (best club out there) has a cute penguin mascot named pebbles 

Answer:
Enigma


'Enigma'

In [9]:
rag_query("Enigma's mascot?")

Question: Enigma's mascot?

Context:
Enigma, (best club out there) has a cute penguin mascot named pebbles 
Kopala guruviah, would snatch your phone and your soul for his daughter. His cabin is AA 105

Answer:
Pebbles


'Pebbles'

In [10]:
rag_query("Where is Kopala's cabin?")

Question: Where is Kopala's cabin?

Context:
Kopala guruviah, would snatch your phone and your soul for his daughter. His cabin is AA 105
Enigma, (best club out there) has a cute penguin mascot named pebbles 

Answer:
AA 105


'AA 105'